In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from scipy import stats
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use(['fast'])
plt.style.use(['seaborn-poster'])
print(plt.style.available)

DATA_FOLDER = "/data/cubes/"
DATA_PATH = os.path.abspath(os.getcwd()) + DATA_FOLDER
invalid_participant = [1,4,11, 12, 18, 39, 53]

['dark_background', 'seaborn-colorblind', 'seaborn-talk', 'seaborn-paper', 'seaborn-deep', 'seaborn-poster', 'fast', 'bmh', 'seaborn-whitegrid', 'ggplot', 'seaborn-ticks', 'seaborn-darkgrid', 'seaborn-pastel', '_classic_test', 'seaborn-notebook', 'classic', 'seaborn-white', 'fivethirtyeight', 'seaborn-dark-palette', 'seaborn-dark', 'seaborn-muted', 'seaborn', 'grayscale', 'seaborn-bright', 'Solarize_Light2']


In [2]:
# Iterate over files and find csv files that have allocation of cubes for each participant
dfs = []
redundant = []
for path in Path(DATA_PATH).rglob('participant_*.csv'): 
    participant_id = int(re.search(r'\d+', path.name).group())
    
    if participant_id in invalid_participant:
        continue
    
    if participant_id not in redundant:
        temp = pd.read_csv(path)
        temp["scene"] = temp["scene"].str[-1:]
        temp["scene"] = temp["scene"].astype('int')
        temp["participant"] = participant_id
        temp["NVC_A"] = False
        temp["NVC_B"] = False

        if int(participant_id) % 2 == 0:
            nvc = "A"
            temp["NVC_A"] = True
        else:
            nvc = "B"
            temp["NVC_B"] = True

        temp["NVC"] = nvc
        temp["allocation_metric"] = (temp["robot_b"] - temp["robot_a"]) / (temp["robot_a"] + temp["robot_b"])
        dfs.append(temp)
        redundant.append(participant_id)
    
raw_data = pd.concat(dfs, axis=0, ignore_index=True)
raw_data["allocation_metric"].fillna(0.0, inplace=True) # handle 0/0
raw_data.head()

,scene,robot_a,robot_b,participant,NVC_A,NVC_B,NVC,allocation_metric
0,0,1,1,40,True,False,A,0.000000
1,1,2,2,40,True,False,A,0.000000
2,2,2,3,40,True,False,A,0.200000
3,3,4,3,40,True,False,A,-0.142857
4,4,2,5,40,True,False,A,0.428571


In [ ]:
odd_participants_data = raw_data[raw_data["participant"] % 2 != 0]
odd_participants_data = odd_participants_data[odd_participants_data["scene"] >= 1]
even_participants_data = raw_data[raw_data["participant"] % 2 == 0]
even_participants_data = even_participants_data[even_participants_data["scene"] >= 1]

In [ ]:
odd_participants_data.head()

In [ ]:
odd_cubes_per_participant = odd_participants_data.groupby("participant").sum()
even_cubes_per_participant = even_participants_data.groupby("participant").sum()

odd_cubes_per_participant = odd_cubes_per_participant.drop(columns=['scene'])
odd_cubes_per_participant = odd_cubes_per_participant.drop(columns=['NVC_A', 'NVC_B'])

even_cubes_per_participant = even_cubes_per_participant.drop(columns=['scene'])
even_cubes_per_participant = even_cubes_per_participant.drop(columns=['NVC_A', 'NVC_B'])


odd_cubes_per_participant['allocation_metric'] = odd_cubes_per_participant['allocation_metric'] / 4

odd_cubes_per_participant['total_cubes'] = (odd_cubes_per_participant['robot_a'] + 
                                                odd_cubes_per_participant['robot_b'])
even_cubes_per_participant['total_cubes'] = (even_cubes_per_participant['robot_a'] + 
                                                even_cubes_per_participant['robot_b'])

odd_cubes_per_participant

In [ ]:
odd_cubes_per_participant.sum()

# Participants which had robotB as NVC

In [ ]:
odd_cubes_per_participant_plot_data = odd_cubes_per_participant.drop(columns=['total_cubes'])
odd_cubes_per_participant_plot_data.plot(kind = 'bar', figsize=(15,6), 
                                         title="The distribution of cubes for each robot when NVC is B.", 
                                         yticks=range(0,25,5))

# Participants which had robotA as NVC

In [ ]:
even_cubes_per_participant.head()

In [ ]:
even_cubes_per_participant.sum()

In [ ]:
even_cubes_per_participant_plot_data = even_cubes_per_participant.drop(columns=['total_cubes'])
even_cubes_per_participant_plot_data.plot(kind = 'bar', figsize=(15,6), title="The distribution of cubes for each robot when NVC is A.", yticks=range(0,50,5))

# NVC-B group by scene

In [ ]:
odd_participants_by_scene = odd_participants_data.groupby("scene").sum().drop(columns=['participant', 'NVC_A', 'NVC_B'])
odd_participants_by_scene

In [ ]:
temp = odd_participants_by_scene
temp['robot a mean'] = odd_participants_by_scene['robot_a']/len(odd_participants_by_scene)
temp['robot b mean'] = odd_participants_by_scene['robot_b']/len(odd_participants_by_scene)
temp

In [ ]:
plt.figure(figsize=(10,6))
plt.xticks(range(1,5))
plt.yticks(range(40,150,5))
plt.scatter(odd_participants_by_scene.index, odd_participants_by_scene["robot_a"], label="A")
plt.scatter(odd_participants_by_scene.index, odd_participants_by_scene["robot_b"], label="B")
plt.xlabel('Scene')
plt.ylabel('Total Cubes')
plt.legend()
plt.show()

# NVC-A group by scene

In [ ]:
even_participants_by_scene = even_participants_data.groupby("scene").sum().drop(columns=['participant']).drop(columns=['NVC_A']).drop(columns=['NVC_B'])
even_participants_by_scene

In [ ]:
temp = even_participants_by_scene
temp['robot a mean'] = even_participants_by_scene['robot_a']/len(even_participants_by_scene)
temp['robot b mean'] = even_participants_by_scene['robot_b']/len(even_participants_by_scene)
temp

In [ ]:
plt.figure(figsize=(10,6))
plt.xticks(range(1,5))
plt.yticks(range(30,150,5))
plt.scatter(even_participants_by_scene.index, even_participants_by_scene["robot_a"], label="A")
plt.scatter(even_participants_by_scene.index, even_participants_by_scene["robot_b"], label="B")
plt.xlabel('Scene')
plt.ylabel('Total Cubes')
plt.legend()
plt.show()

In [ ]:
color = {'boxes': 'DarkGreen', 'whiskers': 'DarkOrange','medians': 'DarkBlue', 'caps': 'Gray'}
ax = even_participants_by_scene.plot(kind = 'box', color = color, sym='', figsize=(8,6), yticks=range(40,150,10),
                                    title = "cubes allocation when NVC is A")
ax.set_ylabel("total cubes")

In [ ]:
color = {'boxes': 'Red', 'whiskers': 'Blue','medians': 'DarkBlue', 'caps': 'Black'}
ax = odd_participants_by_scene.plot(kind = 'box', color = color, sym='', figsize=(8,6),yticks=range(30,150,10),
                                    title = "cubes allocation when NVC is B")
ax.set_ylabel("total cubes ")

In [ ]:
odd_participants_data.groupby(["participant", "scene"]).sum().head()

In [ ]:
odd_participants_data.groupby(["participant", "scene"]).sum().drop(columns=['NVC_A','NVC_B']).plot(kind='bar', figsize=(20,10), by='participant')

In [ ]:
even_participants_data.groupby(["participant", "scene"]).sum().head()

In [ ]:
even_participants_data.groupby(["participant", "scene"]).sum().drop(columns=['NVC_A','NVC_B']).plot(kind='bar', figsize=(20,10))

# Separating data scene wise for participants where NVC is B

In [ ]:
#odds_scene_0 = odd_participants_data[(odd_participants_data["scene"] == 0)]
odds_scene_1 = odd_participants_data[(odd_participants_data["scene"] == 1)]
odds_scene_2 = odd_participants_data[(odd_participants_data["scene"] == 2)]
odds_scene_3 = odd_participants_data[(odd_participants_data["scene"] == 3)]
odds_scene_4 = odd_participants_data[(odd_participants_data["scene"] == 4)]

# Cubes when NVC is B (scene 1 and 2)

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10,10))

ax[0][0].set_ylabel("scene_1")
ax[1][0].set_ylabel("scene_2")
sns.distplot(odds_scene_1["robot_a"], bins=5, ax=ax[0][0])
sns.distplot(odds_scene_1["robot_b"], bins=5, ax=ax[0][1])
sns.distplot(odds_scene_2["robot_a"], bins=5, ax=ax[1][0])
sns.distplot(odds_scene_2["robot_b"], bins=5, ax=ax[1][1])

# Cubes when NVC is B (scene 3 and 4)

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10,10))

ax[0][0].set_ylabel("scene_3")
ax[1][0].set_ylabel("scene_4")
sns.distplot(odds_scene_3["robot_a"], bins=5, ax=ax[0][0])
sns.distplot(odds_scene_3["robot_b"], bins=5, ax=ax[0][1])
sns.distplot(odds_scene_4["robot_a"], bins=5, ax=ax[1][0])
sns.distplot(odds_scene_4["robot_b"], bins=5, ax=ax[1][1])

In [ ]:
stats.normaltest(odds_scene_1["robot_a"])

In [ ]:
stats.normaltest(odds_scene_1["robot_b"])

In [ ]:
stats.normaltest(odds_scene_2["robot_a"])

In [ ]:
stats.normaltest(odds_scene_2["robot_b"])

In [ ]:
stats.normaltest(odds_scene_3["robot_a"])

In [ ]:
stats.normaltest(odds_scene_3["robot_b"])

In [ ]:
stats.normaltest(odds_scene_4["robot_a"])

In [ ]:
stats.normaltest(odds_scene_4["robot_b"])

# Wilcoxon Test

h0: there is no difference between a robot A and robot B <br>
h1: there is a difference

Scene 1

In [ ]:
stats.wilcoxon(odds_scene_1["robot_a"], odds_scene_1["robot_b"])

Scene 2

In [ ]:
stats.wilcoxon(odds_scene_2["robot_a"], odds_scene_2["robot_b"])

Scene 3

In [ ]:
stats.wilcoxon(odds_scene_3["robot_a"], odds_scene_3["robot_b"])

scene 4

In [ ]:
stats.wilcoxon(odds_scene_4["robot_a"], odds_scene_4["robot_b"])

# Conclusion

### For scene 1,2, and 4 p-value > 0.05, accept h0

### For scene 3 p-value < 0.05, reject h0

# Pair t-test

Scene 1

In [ ]:
stats.ttest_rel(odds_scene_1["robot_a"], odds_scene_1["robot_b"])

Scene 2

In [ ]:
stats.ttest_rel(odds_scene_2["robot_a"], odds_scene_2["robot_b"])

Scene 3

In [ ]:
stats.ttest_rel(odds_scene_3["robot_a"], odds_scene_3["robot_b"])

Scene 4

In [ ]:
stats.ttest_rel(odds_scene_4["robot_a"], odds_scene_4["robot_b"])

### Conclusion
#### Scene 1, 2, 4 p-value >= 0.05, accept h0: There is no difference between a robot A and a robot B
#### Scene 3 p-value < 0.05, reject h0: There is a difference between a robot A and a robot B

# Separating data scene wise for participants where NVC is A

In [ ]:
#evens_scene_0 = even_participants_data[(even_participants_data["scene"] == 0)]
evens_scene_1 = even_participants_data[(even_participants_data["scene"] == 1)]
evens_scene_2 = even_participants_data[(even_participants_data["scene"] == 2)]
evens_scene_3 = even_participants_data[(even_participants_data["scene"] == 3)]
evens_scene_4 = even_participants_data[(even_participants_data["scene"] == 4)]

# Cubes when NVC is A (scene 1 and 2)

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10,10))

ax[0][0].set_ylabel("scene_1")
ax[1][0].set_ylabel("scene_2")
sns.distplot(evens_scene_1["robot_a"], bins=5, ax=ax[0][0])
sns.distplot(evens_scene_1["robot_b"], bins=5, ax=ax[0][1])
sns.distplot(evens_scene_2["robot_a"], bins=5, ax=ax[1][0])
sns.distplot(evens_scene_2["robot_b"], bins=5, ax=ax[1][1])

# Cubes when NVC is A (scene 3 and 4)

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10,10))

ax[0][0].set_ylabel("scene_3")
ax[1][0].set_ylabel("scene_4")
sns.distplot(evens_scene_3["robot_a"], bins=5, ax=ax[0][0])
sns.distplot(evens_scene_3["robot_b"], bins=5, ax=ax[0][1])
sns.distplot(evens_scene_4["robot_a"], bins=5, ax=ax[1][0])
sns.distplot(evens_scene_4["robot_b"], bins=5, ax=ax[1][1])

In [ ]:
stats.normaltest(evens_scene_1["robot_a"])

In [ ]:
stats.normaltest(evens_scene_1["robot_b"])

In [ ]:
stats.normaltest(evens_scene_2["robot_a"])

In [ ]:
stats.normaltest(evens_scene_2["robot_b"])

In [ ]:
stats.normaltest(evens_scene_3["robot_a"])

In [ ]:
stats.normaltest(evens_scene_3["robot_b"])

In [ ]:
stats.normaltest(evens_scene_4["robot_a"])

In [ ]:
stats.normaltest(evens_scene_4["robot_b"])

# Wilcoxon Test when NVC A

h0: there is no difference between a robot A and robot B <br>
h1: there is a difference

Scene 1

In [ ]:
stats.wilcoxon(evens_scene_1["robot_a"], evens_scene_1["robot_b"])

Scene 2

In [ ]:
stats.wilcoxon(evens_scene_2["robot_a"], evens_scene_2["robot_b"])

Scene 3

In [ ]:
stats.wilcoxon(evens_scene_3["robot_a"], evens_scene_3["robot_b"])

Scene 4

In [ ]:
stats.wilcoxon(evens_scene_4["robot_a"], evens_scene_4["robot_b"])

# Conclusion

### For scene 1,3, and 4 p-value > 0.05, accept h0

### For scene 2 p-value < 0.05, reject h0

# Pair t-test

Scene 1

In [ ]:
stats.ttest_rel(evens_scene_1["robot_a"], evens_scene_1["robot_b"])

Scene 2

In [ ]:
stats.ttest_rel(evens_scene_2["robot_a"], evens_scene_2["robot_b"])

Scene 3

In [ ]:
stats.ttest_rel(evens_scene_3["robot_a"], evens_scene_3["robot_b"])

Scene 4

In [ ]:
stats.ttest_rel(evens_scene_4["robot_a"], evens_scene_4["robot_b"])

### Conclusion
#### Scene 1, 3, 4 p-value >= 0.05, accept h0: There is no difference between a robot A and a robot B
#### Scene 2 p-value < 0.05, reject h0: There is a difference between a robot A and a robot B

# Separating the data for eye color

In [3]:
swap_heads_id = 26

In [4]:
def separate_red_yellow_eyes_data(data, swap_heads_id):
    red_eye_on_A = data[data['participant'] <= swap_heads_id]
    yellow_eye_on_A = data[data['participant'] > swap_heads_id]
    
    return red_eye_on_A, yellow_eye_on_A

# Separating data for NVC and non NVCs

In [5]:
## separates nv and non nvc data with scene number > scene_num
def separate_NVC_nonNVC(data, scene_num=0):
    odd_participants_data = data[data["participant"] % 2 != 0]
    odd_participants_data = odd_participants_data[odd_participants_data["scene"] > scene_num]
    
    even_participants_data = data[data["participant"] % 2 == 0]
    even_participants_data = even_participants_data[even_participants_data["scene"] > scene_num]
    
    return even_participants_data, odd_participants_data

In [6]:
## takes a list of colums to be dropped and return clean data
def drop_colums(data, column_list):
    clean_data = data.drop(columns=column_list)
    return clean_data

In [7]:
## takes a list of column names and group them
def group_data(data, group_list):
    groupped_data = data.groupby(group_list).sum()
    return groupped_data

# Separating data scene wise

In [8]:
def separate_data_scene_wise(data):
    data_1 = data[(data["scene"] == 1)]
    data_2 = data[(data["scene"] == 2)]
    data_3 = data[(data["scene"] == 3)]
    data_4 = data[(data["scene"] == 4)]
    
    return data_1, data_2, data_3, data_4

In [9]:
#takes data and list of column names
#wilcoxon_test(red_eye, ['robot_a', 'robot_b'])
def wilcoxon_test(data, column_names):
    return stats.wilcoxon(data[column_names[0]], data[column_names[1]])

In [10]:
#takes data and list of column names
#pair_t_test(red_eye, ['robot_a', 'robot_b'])
def pair_t_test(data, column_names):
    return stats.ttest_rel(data[column_names[0]], data[column_names[1]])

In [11]:
##takes two data and compare their same colum  
def wilcoxon_ranksum_test(data1, data2, column):
    return stats.ranksums(data1[column], data2[column])

In [ ]:
cols=['robot_a', 'robot_b']

In [ ]:
print(wilcoxon_test(y_s1, cols))
print(wilcoxon_test(y_s2, cols))
print(wilcoxon_test(y_s3, cols))
print(wilcoxon_test(y_s4, cols))

In [ ]:
print(pair_t_test(y_s1, cols))
print(pair_t_test(y_s2, cols))
print(pair_t_test(y_s3, cols))
print(pair_t_test(y_s4, cols))

In [ ]:
print(wilcoxon_test(r_s1, cols))
print(wilcoxon_test(r_s2, cols))
print(wilcoxon_test(r_s3, cols))
print(wilcoxon_test(r_s4, cols))

In [ ]:
print(pair_t_test(r_s1, cols))
print(pair_t_test(r_s2, cols))
print(pair_t_test(r_s3, cols))
print(pair_t_test(r_s4, cols))

In [ ]:
red_on_A, red_on_B = separate_red_yellow_eyes_data(raw_data, swap_heads_id)

In [ ]:
red_on_B_s1, red_on_B_s2, red_on_B_s3, red_on_B_s4 = separate_data_scene_wise(red_on_B)

In [ ]:
red_on_A_s1, red_on_A_s2, red_on_A_s3, red_on_A_s4 = separate_data_scene_wise(red_on_A)

In [ ]:
red_on_B = drop_colums(red_on_B, ['NVC_A','NVC_B', 'NVC'])

In [ ]:
red_on_B = red_on_B[red_on_B['scene'] > 0]

In [ ]:
red_on_B.groupby("scene").sum().drop(columns=['participant']).plot(kind='bar', title = "total cubes allocation per scene for 27-..")

In [ ]:
red_on_A = drop_colums(red_on_A, ['NVC_A','NVC_B', 'NVC'])
red_on_A = red_on_A[red_on_A['scene'] > 0]
red_on_A.groupby("scene").sum().drop(columns=['participant']).plot(kind='bar', title = "total cubes allocation per scene for 1-26")

In [ ]:
red_on_A = group_data(red_on_A,['participant'])
red_on_A = drop_colums(red_on_A, ['scene'])
red_on_A['allocation_metric'] = red_on_A['allocation_metric'] / 4
red_on_A.head()

In [ ]:
red_on_B = group_data(red_on_B,['participant'])
red_on_B = drop_colums(red_on_B, ['scene'])
red_on_B['allocation_metric'] = red_on_B['allocation_metric'] / 4
red_on_B.head()

In [12]:
ranksum_colum = 'allocation_metric'

In [ ]:
print(wilcoxon_ranksum_test(red_on_A_s1, red_on_B_s1, ranksum_colum))
print(wilcoxon_ranksum_test(red_on_A_s2, red_on_B_s2, ranksum_colum))
print(wilcoxon_ranksum_test(red_on_A_s3, red_on_B_s3, ranksum_colum))
print(wilcoxon_ranksum_test(red_on_A_s4, red_on_B_s4, ranksum_colum))
print(wilcoxon_ranksum_test(red_on_A, red_on_B, ranksum_colum))

In [ ]:
temp = raw_data.copy()

In [13]:
raw_data['Allocation_A']=raw_data['robot_a'] / (raw_data['robot_a'] + raw_data['robot_b'])
raw_data['Allocation_B']=raw_data['robot_b'] / (raw_data['robot_a'] + raw_data['robot_b'])

In [14]:
even, odd = separate_NVC_nonNVC(raw_data)

In [15]:
e_s1, e_s2, e_s3, e_s4 = separate_data_scene_wise(even)
o_s1, o_s2, o_s3, o_s4 = separate_data_scene_wise(odd)

In [16]:
print(wilcoxon_ranksum_test(e_s1, o_s1, ranksum_colum))
print(wilcoxon_ranksum_test(e_s2, o_s2, ranksum_colum))
print(wilcoxon_ranksum_test(e_s3, o_s3, ranksum_colum))
print(wilcoxon_ranksum_test(e_s4, o_s4, ranksum_colum))

RanksumsResult(statistic=-1.6476885412171023, pvalue=0.09941660064420932)
RanksumsResult(statistic=0.3624914790677625, pvalue=0.7169847899212756)
RanksumsResult(statistic=1.0435360761041648, pvalue=0.2967000813888422)
RanksumsResult(statistic=0.6371062359372796, pvalue=0.5240556469270492)


In [17]:
print(wilcoxon_ranksum_test(e_s1, o_s1, 'robot_a'))
print(wilcoxon_ranksum_test(e_s2, o_s2, 'robot_a'))
print(wilcoxon_ranksum_test(e_s3, o_s3, 'robot_a'))
print(wilcoxon_ranksum_test(e_s4, o_s4, 'robot_a'))

RanksumsResult(statistic=1.142397388577191, pvalue=0.25328887626228)
RanksumsResult(statistic=-0.19772262494605228, pvalue=0.8432620870195109)
RanksumsResult(statistic=-1.7026114925910056, pvalue=0.0886407979122168)
RanksumsResult(statistic=-0.27461475686951703, pvalue=0.7836122277349931)


In [18]:
print(wilcoxon_ranksum_test(e_s1, o_s1, 'robot_b'))
print(wilcoxon_ranksum_test(e_s2, o_s2, 'robot_b'))
print(wilcoxon_ranksum_test(e_s3, o_s3, 'robot_b'))
print(wilcoxon_ranksum_test(e_s4, o_s4, 'robot_b'))

RanksumsResult(statistic=-1.8893495272622773, pvalue=0.05884500921534248)
RanksumsResult(statistic=0.8567980414328932, pvalue=0.39155650830559197)
RanksumsResult(statistic=0.45036820126600796, pvalue=0.6524449696074472)
RanksumsResult(statistic=0.5052911526399113, pvalue=0.6133543559547467)


In [19]:
print(wilcoxon_ranksum_test(e_s1, o_s1, 'Allocation_A'))
print(wilcoxon_ranksum_test(e_s2, o_s2, 'Allocation_A'))
print(wilcoxon_ranksum_test(e_s3, o_s3, 'Allocation_A'))
print(wilcoxon_ranksum_test(e_s4, o_s4, 'Allocation_A'))

RanksumsResult(statistic=2.240856416055259, pvalue=0.025035377189050925)
RanksumsResult(statistic=-0.3624914790677625, pvalue=0.7169847899212756)
RanksumsResult(statistic=-0.7689213192346477, pvalue=0.44194002047903236)
RanksumsResult(statistic=-0.6371062359372796, pvalue=0.5240556469270492)


In [20]:
print(wilcoxon_ranksum_test(e_s1, o_s1, 'Allocation_B'))
print(wilcoxon_ranksum_test(e_s2, o_s2, 'Allocation_B'))
print(wilcoxon_ranksum_test(e_s3, o_s3, 'Allocation_B'))
print(wilcoxon_ranksum_test(e_s4, o_s4, 'Allocation_B'))

RanksumsResult(statistic=-0.724982958135525, pvalue=0.4684624853723316)
RanksumsResult(statistic=0.3624914790677625, pvalue=0.7169847899212756)
RanksumsResult(statistic=1.274212471874559, pvalue=0.2025881263524114)
RanksumsResult(statistic=0.6371062359372796, pvalue=0.5240556469270492)


In [21]:
e_s1.shape

(23, 10)

In [22]:
o_s1.shape

(23, 10)

In [23]:
print(stats.wilcoxon(e_s1['Allocation_A'], o_s1['Allocation_A']))
print(stats.wilcoxon(e_s2['Allocation_A'], o_s2['Allocation_A']))
print(stats.wilcoxon(e_s3['Allocation_A'], o_s3['Allocation_A']))
print(stats.wilcoxon(e_s4['Allocation_A'], o_s4['Allocation_A']))

WilcoxonResult(statistic=57.0, pvalue=0.023954977404240428)
WilcoxonResult(statistic=108.0, pvalue=0.5479385211404284)
WilcoxonResult(statistic=77.0, pvalue=0.18044580919518793)
WilcoxonResult(statistic=77.0, pvalue=0.4677166916814839)


/informatik2/students/home/8bhatia/.local/lib/python3.6/site-packages/scipy/stats/morestats.py:2391: RuntimeWarning: invalid value encountered in greater
  r_plus = np.sum((d > 0) * r, axis=0)
/informatik2/students/home/8bhatia/.local/lib/python3.6/site-packages/scipy/stats/morestats.py:2392: RuntimeWarning: invalid value encountered in less
  r_minus = np.sum((d < 0) * r, axis=0)


In [24]:
print(stats.wilcoxon(o_s1['Allocation_B'], e_s1['Allocation_B']))
print(stats.wilcoxon(o_s2['Allocation_B'], e_s2['Allocation_B']))
print(stats.wilcoxon(o_s3['Allocation_B'], e_s3['Allocation_B']))
print(stats.wilcoxon(o_s4['Allocation_B'], e_s4['Allocation_B']))

WilcoxonResult(statistic=57.0, pvalue=0.023954977404240428)
WilcoxonResult(statistic=108.0, pvalue=0.5479385211404284)
WilcoxonResult(statistic=76.5, pvalue=0.17488430101404018)
WilcoxonResult(statistic=78.0, pvalue=0.4928047040675736)


In [26]:
o_s1.head()

,scene,robot_a,robot_b,participant,NVC_A,NVC_B,NVC,allocation_metric,Allocation_A,Allocation_B
6,1,2,3,41,False,True,B,0.200000,0.400000,0.600000
21,1,0,2,31,False,True,B,1.000000,0.000000,1.000000
31,1,3,0,33,False,True,B,-1.000000,1.000000,0.000000
41,1,3,1,35,False,True,B,-0.500000,0.750000,0.250000
51,1,1,2,37,False,True,B,0.333333,0.333333,0.666667


In [27]:
e_s1.head()

,scene,robot_a,robot_b,participant,NVC_A,NVC_B,NVC,allocation_metric,Allocation_A,Allocation_B
1,1,2,2,40,True,False,A,0.000000,0.500000,0.500000
11,1,5,2,42,True,False,A,-0.428571,0.714286,0.285714
16,1,3,1,30,True,False,A,-0.500000,0.750000,0.250000
26,1,3,0,32,True,False,A,-1.000000,1.000000,0.000000
36,1,1,2,34,True,False,A,0.333333,0.333333,0.666667
